<h1>What If Tool</h1>

Get it running

In [1]:
#@title Install the What-If Tool widget if running in colab {display-mode: "form"}

try:
  import google.colab
  !pip install --upgrade witwidget
except:
  pass

In [12]:
#@title Define helper functions {display-mode: "form"}

import pandas as pd
import numpy as np
import tensorflow as tf
import functools

pd.set_option('display.max_columns', None)

# Creates a tf feature spec from the dataframe and columns specified.
def create_feature_spec(df, columns=None):
    feature_spec = {}
    if columns == None:
        columns = df.columns.values.tolist()
    for f in columns:
        if df[f].dtype is np.dtype(np.int64):
            feature_spec[f] = tf.FixedLenFeature(shape=(), dtype=tf.int64)
        elif df[f].dtype is np.dtype(np.float64):
            feature_spec[f] = tf.FixedLenFeature(shape=(), dtype=tf.float32)
        else:
            feature_spec[f] = tf.FixedLenFeature(shape=(), dtype=tf.string)
    return feature_spec

# Creates simple numeric and categorical feature columns from a feature spec and a
# list of columns from that spec to use.
#
# NOTE: Models might perform better with some feature engineering such as bucketed
# numeric columns and hash-bucket/embedding columns for categorical features.
def create_feature_columns(df, columns, feature_spec):
    ret = []
    for col in columns:
        if feature_spec[col].dtype is tf.int64 or feature_spec[col].dtype is tf.float32:
            ret.append(tf.feature_column.numeric_column(col))
        else:
            ret.append(tf.feature_column.indicator_column(
                tf.feature_column.categorical_column_with_vocabulary_list(col, list(df[col].unique()))))
    return ret

# An input function for providing input to a model from tf.Examples
def tfexamples_input_fn(examples, feature_spec, label, mode=tf.estimator.ModeKeys.EVAL,
                       num_epochs=None, 
                       batch_size=64):
    def ex_generator():
        for i in range(len(examples)):
            yield examples[i].SerializeToString()
    dataset = tf.data.Dataset.from_generator(
      ex_generator, tf.dtypes.string, tf.TensorShape([]))
    if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.shuffle(buffer_size=2 * batch_size + 1)
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda tf_example: parse_tf_example(tf_example, label, feature_spec))
    dataset = dataset.repeat(num_epochs)
    return dataset

# Parses Tf.Example protos into features for the input function.
def parse_tf_example(example_proto, label, feature_spec):
    parsed_features = tf.parse_example(serialized=example_proto, features=feature_spec)
    target = parsed_features.pop(label)
    return parsed_features, target

# Converts a dataframe into a list of tf.Example protos.
def df_to_examples(df, columns=None):
    examples = []
    if columns == None:
        columns = df.columns.values.tolist()
    for index, row in df.iterrows():
        example = tf.train.Example()
        for col in columns:
            if df[col].dtype is np.dtype(np.int64):
                example.features.feature[col].int64_list.value.append(int(row[col]))
            elif df[col].dtype is np.dtype(np.float64):
                example.features.feature[col].float_list.value.append(row[col])
            elif row[col] == row[col]:
                example.features.feature[col].bytes_list.value.append(row[col].encode('utf-8'))
        examples.append(example)
    return examples

# Converts a dataframe column into a column of 0's and 1's based on the provided test.
# Used to force label columns to be numeric for binary classification using a TF estimator.
def make_label_column_numeric(df, label_column, test):
  df[label_column] = np.where(test(df[label_column]), 1, 0)

In [20]:
#@title Read training dataset from CSV {display-mode: "form"}

import pandas as pd


sqf = pd.read_csv('nyc2003.csv')
print(sqf.shape)
display(sqf.head())


(69998, 111)


,year,pct,ser_num,datestop,timestop,recstat,inout,trhsloc,perobs,crimsusp,perstop,typeofid,explnstp,othpers,arstmade,arstoffn,sumissue,sumoffen,compyear,comppct,offunif,officrid,frisked,searched,contrabn,adtlrept,pistol,riflshot,asltweap,knifcuti,machgun,othrweap,pf_hands,pf_wall,pf_grnd,pf_drwep,pf_ptwep,pf_baton,pf_hcuff,pf_pepsp,pf_other,radio,ac_rept,ac_inves,rf_vcrim,rf_othsw,ac_proxm,rf_attir,cs_objcs,cs_descr,cs_casng,cs_lkout,rf_vcact,cs_cloth,cs_drgtr,ac_evasv,ac_assoc,cs_furtv,rf_rfcmp,ac_cgdir,rf_verbl,cs_vcrim,cs_bulge,cs_other,ac_incid,ac_time,rf_knowl,ac_stsnd,ac_other,sb_hdobj,sb_outln,sb_admis,sb_other,repcmd,revcmd,rf_furt,rf_bulg,offverb,offshld,sex,race,dob,age,ht_feet,ht_inch,weight,haircolr,eyecolor,build,othfeatr,addrtyp,rescode,premtype,premname,addrnum,stname,stinter,crossst,aptnum,city,state,zip,addrpct,sector,beat,post,xcoord,ycoord,dettypcm,linecm,detailcm
0,2003,70,5,1012003,03:00,,O,,1,ROBBERY,2,V,Y,Y,N,,N,,0,0,Y,,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,Y,N,N,N,N,N,N,N,N,N,N,Y,N,Y,N,N,N,Y,N,N,N,N,N,N,N,N,N,071,071,Y,N,,,M,B,07291986,16,5,9,160,BK,BR,M,,L,,,,240,CROWN ST.,,,,,,,,,,,,,,,
1,2003,70,3,1012003,03:00,,O,,1,ROBBERY,3,V,Y,Y,N,,N,,0,0,Y,,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,Y,N,Y,N,N,Y,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,071,071,N,N,,,M,B,10151988,15,5,8,150,BK,ZZ,M,,L,,,SIDEWALK,240,CROWN ST (F/O),,,,,,,,,,,,,,,
2,2003,68,1406,1012003,03:00,,I,,1,CPW,5,V,Y,Y,N,,N,,0,0,Y,,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,Y,N,N,N,Y,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,071,071,N,N,,,M,Q,08271963,39,5,6,180,BK,BR,M,,L,,,BASEMNT,NaN,270 CLOWN ST,,,,,,,,,,,,,,,
3,2003,48,4,1012003,16:00,,I,,1,BURGLARY,10,R,Y,Y,N,,N,,0,0,N,,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,049,049,N,N,V,S,F,Q,12311900,20,5,6,0,BK,XX,Z,,L,,,BUILDING,2125,CRUJES AVE,,,,,,,,,,,,,,,
4,2003,48,999991,1022003,03:35,,O,,30,ROBBERY,10,P,Y,N,N,,N,,0,0,Y,,Y,N,N,N,N,N,N,N,N,N,Y,Y,N,N,N,N,Y,N,N,N,N,N,N,Y,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,048,319,N,N,,,M,B,06081983,19,6,0,155,BK,BK,T,,L,,,STREET,NaN,I/O WHITE PLAINS RD & ALLERTON,,,,,,,,,,,,,,,


In [21]:
#@title Specify input columns and column to predict {display-mode: "form"}
import numpy as np

# Make up ML scores for now
#   TODO: ML model will take in features of suspect (demographics, reason for stop, location) to predict guilt (i.e. summons or arrest)
#     - ultimately, I'd like to see whether this AI produces more fair outcomes than what the officers chose to do
pass


# Ground turth (i.e. arrest or summons)
def is_guilty(row):
    if row['sumissue']=='Y' or row['arstmade']=='Y':
        return 'Y'
    else:
        return 'N'
    
sqf['guilty'] = sqf.apply(is_guilty, axis=1)


# Officer judgment ('frisk' or 'not frisk'). Following the tutorial, they want this as 0 or 1 (cuz fitting approximator)
label_column = 'frisked'
#label_column = 'guilty'
sqf[label_column] = np.where(sqf[label_column] == 'N', 0, 1)


# Translate features into human-readable
def race_readable(race):
    mapping = {'B':'Black','Q':'White Hispanic','W':'White','Z':'Other','P':'Black Hispanic','A':'Asian / Pacific Islander',
               'X':'Unknown', 'I':'American Indian / Alaskan Native', ' ':'(not listed)'}
    return mapping[race]

def build_readable(build):
    mapping = {' ':'(not listed)', 'H':'Heavy', 'M':'Medium', 'T':'Thin', 'U':'Muscular', 'Z':'Unknown'}
    return mapping[build]

def crime_subset(crime):
    include = ['ASSAULT', 'BURG', 'CRIM TRESPASS', 'CPCS', 'CRIM TRES', 'CRIMINAL TRESPASS', 'ROBBERY', 'BURGLARY', 'GLA', 'CPW']
    if crime in include:
        return crime
    else:
        return 'Other Crime'
    
def premname_subset(premname):
    include = [' ', 'SIDEWALK', 'CRIM TRESPASS', 'CPCS', 'CRIM TRES', 'CRIMINAL TRESPASS', 'ROBBERY', 'BURGLARY', 'GLA', 'CPW']
    if premname in include:
        return premname
    else:
        return 'Other Location'    

def age_to_decade(age_s):
    if age_s == ' ':
        return 0
        return '(null)'
    age = int(age_s[0])
    if age == 0:
        return 0
        return '(null)'
    if age>90:
        return 90
        return '90+'
    return age
    return '%s0-%s9' % (age,age)

    

sqf[    'race'] = sqf[    'race'].apply(race_readable)
sqf[    'build'] = sqf[  'build'].apply(build_readable)
sqf['crimsusp'] = sqf['crimsusp'].apply(crime_subset)
sqf['premname'] = sqf['premname'].apply(premname_subset)
sqf[     'age'] = sqf['age'].apply(age_to_decade)


# After the above filtering, still only load 10,000 datapoints for the WIT
sqf = sqf.iloc[:2000]



# Get list of all columns from the dataset we will use for model input or output.
input_features = ['age', 'sex', 'build', 'race', 'crimsusp', 'premname', 'ac_proxm', 'ac_evasv', 'ac_incid']
#input_features = ['sex', 'age', 'race']
features_and_labels = input_features + [label_column]

features_for_file = input_features + ['guilty', 'frisked']


In [22]:
#@title Convert dataset to tf.Example protos {display-mode: "form"}

examples = df_to_examples(sqf, features_for_file)

In [23]:
#@title Create and train the classifier {display-mode: "form"}

num_steps = 2000  #@param {type: "number"}
tf.logging.set_verbosity(tf.logging.DEBUG)

# Create a feature spec for the classifier
feature_spec = create_feature_spec(sqf, features_and_labels)

# Define and train the classifier
train_inpf = functools.partial(tfexamples_input_fn, examples, feature_spec, label_column)
classifier = tf.estimator.LinearClassifier(
    feature_columns=create_feature_columns(sqf, input_features, feature_spec))
classifier.train(train_inpf, steps=num_steps)

I0927 10:41:14.285858 4548138432 estimator.py:1790] Using default config.
W0927 10:41:14.287024 4548138432 estimator.py:1811] Using temporary folder as model directory: /var/folders/k3/2_2z6mp53kl212qb14t46lqm0000gn/T/tmp2_m8wi9y
I0927 10:41:14.287750 4548138432 estimator.py:209] Using config: {'_model_dir': '/var/folders/k3/2_2z6mp53kl212qb14t46lqm0000gn/T/tmp2_m8wi9y', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1325e3860>, '_tas

I0927 10:41:24.265290 4548138432 basic_session_run_hooks.py:260] loss = 36.87732, step = 901 (0.720 sec)
I0927 10:41:24.980407 4548138432 basic_session_run_hooks.py:692] global_step/sec: 139.596
I0927 10:41:24.981420 4548138432 basic_session_run_hooks.py:260] loss = 38.495598, step = 1001 (0.716 sec)
I0927 10:41:25.787429 4548138432 basic_session_run_hooks.py:692] global_step/sec: 123.92
I0927 10:41:25.789082 4548138432 basic_session_run_hooks.py:260] loss = 40.283585, step = 1101 (0.808 sec)
I0927 10:41:26.583685 4548138432 basic_session_run_hooks.py:692] global_step/sec: 125.581
I0927 10:41:26.584884 4548138432 basic_session_run_hooks.py:260] loss = 41.7607, step = 1201 (0.796 sec)
I0927 10:41:27.382687 4548138432 basic_session_run_hooks.py:692] global_step/sec: 125.157
I0927 10:41:27.383751 4548138432 basic_session_run_hooks.py:260] loss = 37.47411, step = 1301 (0.799 sec)
I0927 10:41:28.095610 4548138432 basic_session_run_hooks.py:692] global_step/sec: 140.267
I0927 10:41:28.096621

In [24]:
#@title Invoke What-If Tool for test data and the trained models {display-mode: "form"}


num_datapoints = 10000  #@param {type: "number"}
tool_height_in_px = 1000  #@param {type: "number"}

from witwidget.notebook.visualization import WitConfigBuilder
from witwidget.notebook.visualization import WitWidget

# Setup the tool with the test examples and the trained classifier
config_builder = WitConfigBuilder(examples[0:num_datapoints]).set_estimator_and_feature_spec(
    classifier, feature_spec)
WitWidget(config_builder, height=tool_height_in_px)

WitWidget(config={'model_type': 'classification', 'label_vocab': [], 'are_sequence_examples': False, 'inferenc…

I0927 10:41:37.688853 4548138432 estimator.py:1145] Calling model_fn.
I0927 10:41:37.755208 4548138432 feature_column_v2.py:2561] Transforming feature_column IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='ac_evasv', vocabulary_list=('N', 'Y'), dtype=tf.string, default_value=-1, num_oov_buckets=0)).
I0927 10:41:37.756105 4548138432 feature_column_v2.py:2561] Transforming feature_column VocabularyListCategoricalColumn(key='ac_evasv', vocabulary_list=('N', 'Y'), dtype=tf.string, default_value=-1, num_oov_buckets=0).
I0927 10:41:37.811451 4548138432 feature_column_v2.py:2561] Transforming feature_column IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='ac_incid', vocabulary_list=('N', 'Y'), dtype=tf.string, default_value=-1, num_oov_buckets=0)).
I0927 10:41:37.813282 4548138432 feature_column_v2.py:2561] Transforming feature_column VocabularyListCategoricalColumn(key='ac_incid', vocabulary_list=('N', 'Y'), dtype=tf.string, default_value=-1, nu